# 웹 크롤링 기본

* 크롤링 사이트: https://www.tripadvisor.co.kr/Restaurants-g294197-Seoul.html
* robots.txt:  https://www.tripadvisor.co.kr/robots.txt


### 크롤링 절차

1. 사이트의 html을 읽어들이기: requests.get(url) 사용
    
2. 텍스트 형태의 데이터를 html 태그별로 구분하여 파싱하기 : BeutifulSoup

3. 특정 태그값만 찾기 : findAll, find

4. 필요한 데이터값 정제하기

5. 데이터 저장하기

In [30]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [31]:
# 크롤링할 웹 사이트 정보
base_url = 'https://www.tripadvisor.co.kr'
seoul_url = '/Restaurants-g294197-Seoul.html'

# get html
response = requests.get(base_url+seoul_url)
response

<Response [200]>

In [32]:
# response.text

In [33]:
# BeautifulSoup을 활용하여 데이터 파싱
soup = BeautifulSoup(response.text, 'html.parser')

In [36]:
# 맛집 목록 가져오기
# <div class="wQjYiB7z"><span><a href="/Restaurant_Review-g294197-d8472275-Reviews-Choigozip_Hongdae-Seoul.html" class="_15_ydu6b" target="_blank">8<!-- -->. <!-- -->최고집 홍대점</a></span></div>
lists = soup.findAll('div', {'class' : 'wQjYiB7z'}) 

# 이미지는 이렇게는 못받아옴. selenium 이용해야함
# lists = soup.findAll('div', {'class' : '_2Hb-Mt7l'}) 
# lists

[]

In [29]:
for item in lists:
#     temp = str(item)
#     print(temp)
#     name = item.text
#     print(name)

#     name = item.text.split('.')
    
    # 광고 외의 식당이름만 뽑기
#     if(len(name) == 2):
#         print(name[0] + name[1])
    
    # 광고만 뽑기
#     if(len(name)<2):
#         print(name[0])

    # 링크 가져오기
    hrefs = item.find('a')
    href = hrefs.get('href')
    full_url = base_url + href
#     print(full_url)

In [21]:
# 데이터 파싱하기
rest_id_list = []
rest_name_list = []
rest_url_list = []

for item in lists:
    # 링크 가져오기
    hrefs = item.find('a')
    href = hrefs.get('href')
    full_url = base_url + href

    # 순위와 이름 분리하기, 광고 제거하기
    #[1 , 장생건강원]
    #[2,  무슨음식점]
    #[광고음식점]
    
    name = item.text.split('.')

    if len(name) == 2:
        rest_id = name[0]
        rest_name = name[1]
        rest_url = full_url
        #데이터 프레임을 만들기 위한 리스트
        rest_id_list.append(rest_id)
        rest_name_list.append(rest_name)
        rest_url_list.append(rest_url)
#         print('[ID]' + rest_id + ' [NAME]' + rest_name + ' [URL]' + rest_url)
        
result = pd.DataFrame(data={'[ID]' : rest_id_list, '[NAME]' : rest_name_list, '[URL]' : rest_url_list})
result.head()
# print(result)
# result

,[ID],[NAME],[URL]
0,1,장생건강원,https://www.tripadvisor.co.kr/Restaurant_Revie...
1,2,플레이버즈,https://www.tripadvisor.co.kr/Restaurant_Revie...
2,3,구스토 타코,https://www.tripadvisor.co.kr/Restaurant_Revie...
3,4,지화자,https://www.tripadvisor.co.kr/Restaurant_Revie...
4,5,853,https://www.tripadvisor.co.kr/Restaurant_Revie...


In [ ]:
# (실습1) 추천글

In [22]:
# (실습2) 크롤링할 웹 사이트 정보
# 'https://www.tripadvisor.co.kr/Restaurants-g297885-Jeju_Jeju_Island.html'

# 제주 맛집 정보 가져오기
jeju_url = '/Restaurants-g297885-Jeju_Jeju_Island.html'

response = requests.get(base_url + jeju_url)
response

<Response [200]>

In [25]:
# BeautifulSoup을 활용하여 데이터 파싱
soup = BeautifulSoup(response.text, 'html.parser')
# soup

In [28]:
# 제주 맛집 이름, url 링크, 리뷰 수, 대표 리뷰 가져오기
list_nm = []
list_url = []
list_rv = []
list_rrv = []

list = soup.findAll('div', {'class': 'wQjYiB7z'})

for item in list:
    item.find
#        print(item.text)
    name = item.text.split('. ')
    
    if (len(name) == 2):
        

1. 인디언키친
2. 명진전복
3. 우진해장국
4. 블루버드 바이 맥파이
5. 마초 그릴
6. 올래국수
7. 고집돌우럭
8. 광원
9. 삼다정
10. 자매국수
11. 돈사돈
12. 협재화덕 도나토스
13. 바그다드
14. 스시 호시카이
15. 흑돈가 제주본점
16. 모디카
17. 고집돌우럭 - 함덕점
18. 애월리안
19. 삼대 국수회관
20. 르 쉬느아
21. 비원
22. 삼성혈해물탕
23. 리치망고
24. 자매국수
25. 화로향
26. 저팔계깡통연탄구이
27. 라아바나
28. 산호 전복
29. 곰막
30. 국수마당
